In [1]:
import gym
import numpy as np

from lttb import LTTB

from tqdm import trange

N, I, O, T = 100, 4, 2, 100
shape = (N, I, O, T)

dt = .001
tau_m = 6. * dt
tau_s = 5. * dt
tau_ro = 8. * dt
tau_star = 2 * dt

sigma_teach = 2.0
sigma_input = 15.0
dv = 1 / 500.
alpha = .005
alpha_rout = .01
Vo = -4
h = -4
s_inh = 20

sigma_Jrec = 2.
sigma_Jout = 0.1

# Here we build the dictionary of the simulation parameters
par = {'dt' : dt, 'tau_m' : tau_m, 'tau_s' : tau_s, 'tau_ro' : tau_ro, 'tau_star' : tau_star,
	   'dv' : dv, 'Vo' : Vo, 'h' : h, 's_inh' : s_inh,
	   'N' : N, 'T' : T, 'I' : I, 'O' : O, 'shape' : shape, 
       'sigma_Jrec' : sigma_Jrec, 'sigma_Jout' : sigma_Jout,
       'alpha' : alpha, 'alpha_rout' : alpha_rout,
	   'sigma_input' : sigma_input, 'sigma_teach' : sigma_teach}

# Here we init our (recurrent) agent
lttb = LTTB (par)



In [2]:
# one time step of the dynamics... to be developed

lttb.step()

MATLAB code to be converted below